# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 53 new papers today
          21 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = get_markdown_badge(paper_id) + " _" + paper['comments'] + "_"
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/20 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2204.06000


extracting tarball to tmp_2204.06000...

 done.
Retrieving document from  https://arxiv.org/e-print/2204.06001


/opt/hostedtoolcache/Python/3.9.12/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3369: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.12/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3369: LatexWarning: Found documentclass in tmp_2204.06000/main.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.12/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:292: LatexWarning: Latex injecting: 'table3_data' from 'tmp_2204.06000/table3_data.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.12/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:292: LatexWarning: Latex injecting: 'table3_data' from 'tmp_2204.06000/Pang et al. 2022 3d OC/table3_data.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.12/x64/lib/pytho

...

Retrieving document from  https://arxiv.org/e-print/2204.06423


extracting tarball to tmp_2204.06423...

 done.
Retrieving document from  https://arxiv.org/e-print/2204.06503


extracting tarball to tmp_2204.06503...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2204.06393-b31b1b.svg)](https://arxiv.org/abs/arXiv:2204.06393) | **A dusty compact object bridging galaxies and quasars at cosmic dawn**  |
|| S. Fujimoto, et al. -- incl., <mark>F. Walter</mark> |
|*Comments*| *49 pages, 15 figures, 2 tables. Authors' version. Published in the 14 April issue of Nature*|
|**Abstract**| Understanding how super-massive black holes form and grow in the early Universe has become a major challenge since the discovery of luminous quasars only 700 million years after the Big Bang. Simulations indicate an evolutionary sequence of dust-reddened quasars emerging from heavily dust-obscured starbursts that then transition to unobscured luminous quasars by expelling gas and dust. Although the last phase has been identified out to a redshift of 7.6, a transitioning quasar has not been found at similar redshifts owing to their faintness at optical and near-infrared wavelengths. Here we report observations of an ultraviolet compact object, GNz7q, associated with a dust-enshrouded starburst at a redshift of z=7.1899+/-0.0005. The host galaxy is more luminous in dust emission than any other known object at this epoch, forming 1,600 solar masses of stars per year within a central radius of 480 parsec. A red point source in the far-ultraviolet is identified in deep, high-resolution imaging and slitless spectroscopy. GNz7q is extremely faint in X-rays, which indicates the emergence of a uniquely ultraviolet compact star-forming region or a Compton-thick super-Eddington black-hole accretion disk at the dusty starburst core. In the latter case, the observed properties are consistent with predictions from cosmological simulations and suggest that GNz7q is an antecedent to unobscured luminous quasars at later epochs. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2204.06000-b31b1b.svg)](https://arxiv.org/abs/arXiv:2204.06000) | **3D Morphology of Open Clusters in the Solar Neighborhood with Gaia EDR3  II: Hierarchical Star Formation Revealed by Spatial and Kinematic  Substructures**  |
|| Xiaoying Pang, et al. -- incl., <mark>Teng Zhang</mark> |
|*Comments*| *Accepted to ApJ, 33 pages, 14 figures. OC 3D interactive visualization included on this http URL*|
|**Abstract**| We identify members of 65 open clusters in the solar neighborhood using the machine-learning algorithm StarGO based on Gaia EDR3 data. After adding members of twenty clusters from previous studies (Pang et al. 2021a,b; Li et al. 2021) we obtain 85 clusters, and study their morphology and kinematics. We classify the substructures outside the tidal radius into four categories: filamentary (f1) and fractal (f2) for clusters $<100$ Myr, and halo (h) and tidal-tail (t) for clusters $>100$ Myr. The kinematical substructures of f1-type clusters are elongated; these resemble the disrupted cluster Group X. Kinematic tails are distinct in t-type clusters, especially Pleiades. We identify 29 hierarchical groups in four young regions (Alessi 20, IC 348, LP 2373, LP 2442); ten among these are new. The hierarchical groups form filament networks. Two regions (Alessi 20, LP 2373) exhibit global "orthogonal" expansion (stellar motion perpendicular to the filament), which might cause complete dispersal. Infalling-like flows (stellar motion along the filament) are found in UBC 31 and related hierarchical groups in the IC 348 region. Stellar groups in the LP 2442 region (LP 2442 gp 1-5) are spatially well-mixed but kinematically coherent. A merging process might be ongoing in the LP 2442 subgroups. For younger systems ($\lesssim30$ Myr), the mean axis ratio, cluster mass and half-mass radius tend to increase with age values. These correlations between structural parameters may imply two dynamical processes occurring in the hierarchical formation scenario in young stellar groups: (1) filament dissolution and (2) sub-group mergers. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2204.06001-b31b1b.svg)](https://arxiv.org/abs/arXiv:2204.06001) | **Constraining Primordial Black Holes using Fast Radio Burst  Gravitational-Lens Interferometry with CHIME/FRB**  |
|| Calvin Leung, et al. -- incl., <mark>Zarif Kader</mark>, <mark>Bradley W. Meyers</mark>, <mark>Kendrick M. Smith</mark> |
|*Comments*| *20 pages, 5 figures*|
|**Abstract**| Fast radio bursts (FRBs) represent an exciting frontier in the study of gravitational lensing, due to their brightness, extragalactic nature, and the compact, coherent characteristics of their emission. In a companion work [Kader, Leung+2022], we use a novel interferometric method to search for gravitationally lensed FRBs in the time domain using bursts detected by CHIME/FRB. There, we dechannelize and autocorrelate electric field data at a time resolution of 1.25 ns. This enables a search for FRBs whose emission is coherently deflected by gravitational lensing around a foreground compact object such as a primordial black hole (PBH). Here, we use our non-detection of lensed FRBs to place novel constraints on the PBH abundance outside the Local Group. We use a novel two-screen model to take into account decoherence from scattering screens in our constraints. Our constraints are subject to a single astrophysical model parameter -- the effective distance between an FRB source and the scattering screen, for which we adopt a fiducial distance of 1 parsec. We find that coherent FRB lensing is a sensitive probe of sub-solar mass compact objects. Having observed no lenses in $172$ bursts from $114$ independent sightlines through the cosmic web, we constrain the fraction of dark matter made of compact objects, such as PBHs, to be $f \lesssim 0.8$, if their masses are $\sim 10^{-3} M_{\odot}$. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2204.06301-b31b1b.svg)](https://arxiv.org/abs/arXiv:2204.06301) | **Estimation of stellar atmospheric parameters from LAMOST DR8  low-resolution spectra with 20$\leq$SNR$<$30**  |
|| <mark>Xiangru Li</mark>, et al. |
|*Comments*| *16 pages, 6 figures, 4 tables*|
|**Abstract**| The accuracy of the estimated stellar atmospheric parameter decreases evidently with the decreasing of spectral signal-to-noise ratio (SNR) and there are a huge amount of this kind observations, especially in case of SNR$<$30. Therefore, it is helpful to improve the parameter estimation performance for these spectra and this work studied the ($T_\texttt{eff}, \log~g$, [Fe/H]) estimation problem for LAMOST DR8 low-resolution spectra with 20$\leq$SNR$<$30. We proposed a data-driven method based on machine learning techniques. Firstly, this scheme detected stellar atmospheric parameter-sensitive features from spectra by the Least Absolute Shrinkage and Selection Operator (LASSO), rejected ineffective data components and irrelevant data. Secondly, a Multi-layer Perceptron (MLP) method was used to estimate stellar atmospheric parameters from the LASSO features. Finally, the performance of the LASSO-MLP was evaluated by computing and analyzing the consistency between its estimation and the reference from the APOGEE (Apache Point Observatory Galactic Evolution Experiment) high-resolution spectra. Experiments show that the Mean Absolute Errors (MAE) of $T_\texttt{eff}, \log~g$, [Fe/H] are reduced from the LASP (137.6 K, 0.195 dex, 0.091 dex) to LASSO-MLP (84.32 K, 0.137 dex, 0.063 dex), which indicate evident improvements on stellar atmospheric parameter estimation. In addition, this work estimated the stellar atmospheric parameters for 1,162,760 low-resolution spectra with 20$\leq$SNR$<$30 from LAMOST DR8 using LASSO-MLP, and released the estimation catalog, learned model, experimental code, trained model, training data and test data for scientific exploration and algorithm study. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2204.06330-b31b1b.svg)](https://arxiv.org/abs/arXiv:2204.06330) | **Effects of stellar-mass primordial black holes on first star formation**  |
|| Boyuan Liu, <mark>Saiyang Zhang</mark>, Volker Bromm |
|*Comments*| *21 pages, 18+3 figures, submitted to MNRAS*|
|**Abstract**| We use cosmological hydrodynamic zoom-in simulations and semi-analytical models to study the effects of primordial black holes (PBHs) on first star formation. Our models self-consistently combine two competing effects: initial (isocurvature) perturbations induced by PBHs and BH accretion feedback. Focusing on PBHs with masses $\sim 30\ \rm M_{\odot}$, we find that the standard picture of first star formation in molecular-cooling minihaloes is not changed by PBHs, as the simulated star-forming gas clouds in the central parsec are very similar to those in the $\rm \Lambda CDM$ case when PBHs make up $f_{\rm PBH}\sim 10^{-4}-0.1$ of dark matter. With a dynamical friction timescale of $\sim 2-10\ \rm Myr$ when the central gas density reaches $10^{5}\ \rm cm^{-3}$, it is also unlikely that PBHs can sink into star-forming discs and affect the evolution of protostars, although they may interact with the stars during the main-sequence stage. At larger scales, PBHs tend to shift star formation to more massive haloes, and accelerate structure formation. The latter effect is stronger in regions with higher initial overdensities. For $f_{\rm PBH}\sim 10^{-4}-0.01$ (allowed by observational constraints), the collapsed mass fraction of haloes hosting Population III stars is similar (within a factor of $\sim2$ at $z\lesssim 30$) to that in $\rm \Lambda CDM$, implying that the impact of stellar-mass PBHs on the cosmic star formation history at $z\gtrsim 10$ is small. We also find that the Lyman-Werner photons from PBH accretion in atomic-cooling haloes may facilitate the formation of direct-collapse BHs. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2204.06423-b31b1b.svg)](https://arxiv.org/abs/arXiv:2204.06423) | **Comparisons of Type Ia Supernova Light Curves in the UV and Optical with  the Swift Ultra-Violet/Optical Telescope**  |
|| Yaswant Devarakonda, <mark>Peter J. Brown</mark> |
|*Comments*| *16 pages, 11 figures. Accepted for publication in AJ*|
|**Abstract**| We examine the light curve parameters of 97 nearby Type Ia supernovae in the ultraviolet and optical using observations from the Swift Ultra-Violet/Optical Telescope. Our light curve models used a linear combinations of templates, which were based on Functional Principal Component Analysis of the optical photometry of SNe Ia. The weights for each principal component used in the fit capture certain aspects of the light curve properties. We find that there is little difference in the overall variability of principal component scores across filters. We also find correlations between certain filters and PC components, indicating that the UV and optical properties seem to be tied together. This is a promising step in UV SNe Ia analysis, and suggests that UV light curves may be used to infer optical properties, paving the way for future cosmological studies. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2204.06503-b31b1b.svg)](https://arxiv.org/abs/arXiv:2204.06503) | **Temperature Structures Associated with Different Components of the  Atmospheric Circulation on Tidally Locked Exoplanets**  |
|| <mark>Neil T. Lewis</mark>, Mark Hammond |
|*Comments*| *Submitted to AAS journals. Comments welcome and appreciated*|
|**Abstract**| Observations of time-resolved thermal emission from tidally locked exoplanets can tell us about their atmospheric temperature structure. Telescopes such as JWST and ARIEL will improve the quality and availability of these measurements. This motivates an improved understanding of the processes that determine atmospheric temperature structure, particularly atmospheric circulation. The circulation is important in determining atmospheric temperatures, not only through its ability to transport heat, but also because any circulation pattern needs to be balanced by horizontal pressure contrasts, therefore implying a particular temperature structure. In this work, we show how the global temperature field on a tidally locked planet can be decomposed into contributions that are balanced by different components of the atmospheric circulation. These are the superrotating jet, stationary Rossby waves, and the divergent circulation. To achieve this, we partition the geopotential field into components balanced by the divergent circulation and the rotational circulation, with the latter comprising the jet and Rossby waves. The partitioned geopotential then implies a corresponding partitioning of the temperature via the hydrostatic relation. We apply these diagnostics to idealised general circulation model simulations, to show how the separate rotational and divergent circulations together make up the total three-dimensional atmospheric temperature structure. We also show how each component contributes distinct signatures to the thermal phase curve of a tidally locked planet. We conclude that this decomposition is a physically meaningful separation of the temperature field that explains its global structure, and can be used to fit observations of thermal emission. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2204.06376-b31b1b.svg)](https://arxiv.org/abs/arXiv:2204.06376) | **A quarter century of spectroscopic monitoring of the nearby M dwarf Gl  514. A super-Earth on an eccentric orbit moving in and out of the habitable  zone**  |
|| M. Damasso, et al. -- incl., <mark>J. I. González Hernández</mark>, <mark>Th. Henning</mark> |
|*Comments*| *34 pages, accepted for publication in Astronomy & Astrophysics*|
|**Abstract**| We investigated the presence of planetary companions around the nearby (7.6 pc) and bright ($V=9$ mag) early-type M dwarf Gl 514, analysing 540 radial velocities collected over nearly 25 years with the HIRES, HARPS, and CARMENES spectrographs. The data are affected by time-correlated signals at the level of 2-3 ms$^{-1}$ due to stellar activity, that we filtered out testing three different models based on Gaussian process regression. As a sanity cross-check, we repeated the analyses using HARPS radial velocities extracted with three different algorithms. We used HIRES radial velocities and Hipparcos-Gaia astrometry to put constraints on the presence of long-period companions, and we analysed TESS photometric data. We found strong evidence that Gl 514 hosts a super-Earth on a likely eccentric orbit, residing in the conservative habitable zone for nearly $34\%$ of its orbital period. The planet Gl 514 b has minimum mass $m_b\sin i_b=5.2\pm0.9$ $M_{\rm Earth}$, orbital period $P_b=140.43\pm0.41$ days, and eccentricity $e_b=0.45^{+0.15}_{-0.14}$. No evidence for transits is found in the TESS light curve. There is no evidence for a longer period companion in the radial velocities and, based on astrometry, we can rule out a $\sim0.2$ $M_{\rm Jup}$ planet at a distance of $\sim3-10$ au, and massive giant planets/brown dwarfs out to several tens of au. We discuss the possible presence of a second low-mass companion at a shorter distance from the host than Gl 514 b. Gl 514 b represents an interesting science case to study the habitability of planets on eccentric orbits. We advocate for additional spectroscopic follow-up to get more accurate and precise planetary parameters. Further follow-up is also needed to investigate sub \ms and shorter period signals. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error [Errno 2] No such file or directory: 'gs'</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2204.06393.md
    + _build/html/tmp_2204.06393/./Mbh-SFR_v9.png
    + _build/html/tmp_2204.06393/./HST_SED_v9.png
    + _build/html/tmp_2204.06393/./Full_SED_v6.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\hi}{\mbox{H {\scshape i}} }$
$\newcommand{\hii}{\mbox{H {\scshape ii}} }$
$\newcommand{\oiii}{[O {\sc iii}]}$
$\newcommand{\oii}{[O {\sc ii}]}$
$\newcommand{\cii}{[C {\sc ii}]}$
$\newcommand{\ci}{[C {\sc i}](2-1)}$
$\newcommand{\nii}{[N {\sc ii}]}$
$\newcommand{\ciii}{C {\sc iii}]}$
$\newcommand{\civ}{C {\sc iv}}$
$\newcommand{\siiv}{Si {\sc iv}}$
$\newcommand{\lya}{Ly\alpha}$
$\newcommand{\mum}{\mum}$
$\newcommand{\dv}{\Delta v_{\rm Ly\alpha}}$
$\newcommand{\ew}{EW_{\rm 0}}$
$\newcommand{\lsun}{L_{\rm \odot}}$
$\newcommand{\msun}{M_{\rm \odot}}$
$\newcommand{\ltir}{L_{\rm TIR}}$
$\newcommand{\nhi}{N_{\rm HI}}$
$\newcommand{\loiii}{L_{\rm[OIII]}}$
$\newcommand{\llya}{L_{\rm Ly\alpha}}$
$\newcommand{\luv}{L_{\rm UV}}$
$\newcommand{\zph}{z_{\rm ph}}$
$\newcommand{\muv}{M_{\rm UV}}$
$\newcommand{\td}{T_{\rm d}}$
$\newcommand{\bd}{\beta_{\rm d}}$
$\newcommand{\md}{M_{\rm dust}}$
$\newcommand{\zoiii}{z_{\rm[OIII]}}$
$\newcommand{\zlya}{z_{\rm Ly\alpha}}$
$\newcommand{\zsun}{Z_{\rm \odot}}$
$\newcommand{\mdyn}{M_{\rm dyn}}$
$\newcommand{\mgas}{M_{\rm gas}}$
$\newcommand{\mmol}{M_{\rm H_{\rm 2}}}$
$\newcommand{\matom}{M_{\rm H_{\rm I}}}$
$\newcommand{\lir}{L_{\rm IR}}$
$\newcommand{\lirs}{L_{\rm IR,SF}}$
$\newcommand{\lira}{L_{\rm IR,AGN}}$
$\newcommand{\targname}{GNz7q}$
$\newcommand{\tcr}{\textcolor{black}}$
$\newcommand{\tcb}{\textcolor{black}}$
$\newcommand{\tcm}{\textcolor{magenta}}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\hi$}{\mbox{H {\scshape i}} }$
$\newcommand{$\hi$i}{\mbox{H {\scshape ii}} }$
$\newcommand{$\oiii$}{[O {\sc iii}]}$
$\newcommand{$\oii$}{[O {\sc ii}]}$
$\newcommand{$\cii$}{[C {\sc ii}]}$
$\newcommand{$\ci$}{[C {\sc i}](2-1)}$
$\newcommand{$\nii$}{[N {\sc ii}]}$
$\newcommand{$\cii$i}{C {\sc iii}]}$
$\newcommand{$\ci$v}{C {\sc iv}}$
$\newcommand{$\siiv$}{Si {\sc iv}}$
$\newcommand{$\lya$}{Ly\alpha}$
$\newcommand{$\mum$}{$\mum$}$
$\newcommand{$\dv$}{\Delta v_{\rm Ly\alpha}}$
$\newcommand{$\ew$}{EW_{\rm 0}}$
$\newcommand{$\lsun$}{L_{\rm \odot}}$
$\newcommand{$\msun$}{M_{\rm \odot}}$
$\newcommand{$\ltir$}{L_{\rm TIR}}$
$\newcommand{$\nhi$}{N_{\rm HI}}$
$\newcommand{$\loiii$}{L_{\rm[OIII]}}$
$\newcommand{$\llya$}{L_{\rm Ly\alpha}}$
$\newcommand{$\luv$}{L_{\rm UV}}$
$\newcommand{$\zph$}{z_{\rm ph}}$
$\newcommand{$\muv$}{M_{\rm UV}}$
$\newcommand{$\td$}{T_{\rm d}}$
$\newcommand{$\bd$}{\beta_{\rm d}}$
$\newcommand{$\md$}{M_{\rm dust}}$
$\newcommand{$\zoiii$}{z_{\rm[OIII]}}$
$\newcommand{$\zlya$}{z_{\rm Ly\alpha}}$
$\newcommand{$\zsun$}{Z_{\rm \odot}}$
$\newcommand{$\md$yn}{M_{\rm dyn}}$
$\newcommand{$\mgas$}{M_{\rm gas}}$
$\newcommand{$\mmol$}{M_{\rm H_{\rm 2}}}$
$\newcommand{$\matom$}{M_{\rm H_{\rm I}}}$
$\newcommand{$\lir$}{L_{\rm IR}}$
$\newcommand{$\lir$s}{L_{\rm IR,SF}}$
$\newcommand{$\lir$a}{L_{\rm IR,AGN}}$
$\newcommand{$\targname$}{GNz7q}$
$\newcommand{$\tcr$}{\textcolor{black}}$
$\newcommand{$\tcb$}{\textcolor{black}}$
$\newcommand{$\tcm$}{\textcolor{magenta}}$</div>



<div id="title">

# A dusty compact object bridging galaxies and quasars at cosmic dawn

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2204.06393-b31b1b.svg)](https://arxiv.org/abs/2204.06393) _49 pages, 15 figures, 2 tables. Authors' version. Published in the 14 April issue of Nature_

</div>
<div id="authors">

S. Fujimoto, et al. -- incl., <mark><mark>F. Walter</mark></mark>

</div>
<div id="abstract">

**Abstract:** Understanding how super-massive black holes form and grow in the early Universe has become a major challenge$\ci$te{volonteri2012,inayoshi2020}since the discovery of luminous quasars only 700 million years after the Big Bang$\ci$te{mortlock2011, banados2018}.Simulations indicate an evolutionary sequence of dust-reddened quasars emerging from heavily dust-obscured starbursts that then transition to unobscured luminous quasars by expelling gas and dust$\ci$te{hopkins2008}.Although the last phase has been identified out to a redshift of 7.6,$\ci$te{wang2021}a transitioning quasar has not been found at similar redshifts owing to their faintness at optical and near-infrared wavelengths. Here we report observations of an ultraviolet compact object,$\targname$, associated with a dust-enshrouded starburst at a redshift of\boldmath$ z=7.1899\pm0.0005$.The host galaxy ismore luminous in dust emission than any other known object at this epoch, forming 1,600 solar masses of stars per year within a centralradius of 480 parsec. A red point source in the far-ultraviolet is identified in deep, high-resolution imaging and slitless spectroscopy.$\targname$is extremely faint in X-rays, which indicates the emergence of a uniquely ultraviolet compact star-forming region or a Compton-thick super-Eddington black-hole accretion disk at the dusty starburst core.In the latter case, the observed properties are consistent with predictions from cosmological simulations$\ci$te{ginolfi2019}and suggest that$\targname$is an antecedent to unobscured luminous quasars at later epochs.

</div>

<div id="div_fig1">

<img src="tmp_2204.06393/./Mbh-SFR_v9.png" alt="Fig4" width="100%"/>

**Figure 4. -** \small\textbf{$|$
SFR and $M_{\rm BH}$ relations for progenitors of luminous quasars in a cosmological simulation}.
\textbf{a.} SFR and BH masses predicted by the semi-analytical model GQd$\ci$te{valiante2016} for selected $z = 7.2$ progenitors of a luminous quasar at $z=6.4$.
Each system is represented by a circle colour-coded by the dark matter halo mass ($M_{\rm DM}$).
The four black circles mark systems that have X-ray, optical, and host galaxy properties similar to $\targname$(See Methods).
\textbf{b.}
BH mass assembly history for the systems marked with the black circles in panel {\bf a}.
At $z = 7.2$, one of them has already grown to $4.0\times10^{7} M_{\odot}$(red circle in panel a) and thereafter it continues to grow by gas accretion and mergers with other BHs to form a SMBH of $M_{\rm BH}=2.5\times10^{8} M_{\odot}$ at $z=6.4$(red line).
The other systems also have relatively low-mass BHs, down to $\sim10^{6.5-7.5} M_{\odot}$ at $z=7.2$(grey lines), but these systems undergo mergers with galaxies hosting more massive BHs.
As a result, these BHs are not the most massive progenitors of the final SMBH, which grows to $\sim10^{9.3-10}$$M_{\odot}$ by $z=6.4$ through gas accretion and mergers with other BH progenitors (grey dashed line).
The black vertical line indicates the redshift of $\targname$.
\label{fig:simulation} (*fig:simulation*)

</div>
<div id="div_fig2">

<img src="tmp_2204.06393/./HST_SED_v9.png" alt="Fig1" width="100%"/>

**Figure 1. -** \small$\tcb${
\textbf{$|$ \boldmath \emph{Hubble Space Telescope} near-infrared images and spectrum of $\targname$.}
The spectrum and photometry show a strong Lyman break at $\lambda_\mathrm{obs}\sim1.0 \mu\mathrm{m}$.
The top panels show the \textit{HST} image cutouts ($5"\times5"$).
The source is unresolved in all deep \textit{HST} images up to the reddest filter available at $1.6 \mu\mathrm{m}$ (WFC3/IR F160W).
In the bottom panel, the black squares and gray dots respectively show the broadband photometry and the slitless spectrum binned by a factor of 4 relative to the nominal pixel scale.
The error bars denote 1$\sigma$ uncertainties.
The labeled black bars indicate the expected wavelengths for the main emission lines based on the $\cii$ 158-$\mu$m line redshift of $\targname$ at $z=7.1899$.
The blue curve represents a composite spectrum of SDSS optically luminous blue quasars$\ci$te{selsing2016} at $1<z<2$, while the red curve shows a red quasar at $z=3.11$ ($\tcb${SDSS spec-6839-56425-146})$\ci$te{sdss_dr12} whose FUV spectrum resembles that of $\targname$.
Both of the lower-$z$ quasar spectra are shifted to $z=7.1899$, normalized at 1.2 $\mu$m, and binned to the same spectral resolution as the $\targname$ spectrum.
The large open circles show the quasar templates integrated through the \textit{HST} filter passbands.
The bandpasses of the ACS/F850LP and WFC3/F105W filters shown at the bottom straddle the spectral break, explaining the faint detection in the former and the suppressed flux density relative to the continuum in the latter.}\label{fig:grism} (*fig:grism*)

</div>
<div id="div_fig3">

<img src="tmp_2204.06393/./Full_SED_v6.png" alt="Fig2" width="100%"/>

**Figure 2. -** \small$\tcb${
\textbf{$|$
\boldmath
The spectral energy distribution of $\targname$ from optical to radio wavelengths.}
Photometry is shown for data from \textit{HST} (0.8--1.6$\mu$m), \textit{Spitzer} (3.6--24$\mu$m), \textit{Herschel} (80--500$\mu$m), JCMT (450 and 850$\mu$m), NOEMA (1 and 3 mm) and VLA (3 and 20 cm) in the GOODS-North field (Extended Data Table 1).}
Triangles indicate 3$\sigma$ upper limits.
The sum of the best-fit quasar/AGN (black solid) and galaxy (black dashed) templates is shown as a red curve.
The radio detection at 20 cm is consistent with the enormous implied SFR of the host galaxy (see Methods).
For comparison, we also show the SEDs of other source populations at similar redshifts: optically-luminous blue quasars at $z=7.54$(J1342+0928$\ci$te{banados2018}; blue squares) and $z=7.08$(J1120+0641$\ci$te{mortlock2011}; blue stars), and a dusty starburst at $z=6.90$(SPT0311-58W$\ci$te{marrone2018}; orange circles).
The blue curve is drawn with the quasar/AGN template normalized to J1120+6410's rest-frame UV emission.
The orange curve is the best-fit SED for SPT0311-58W, taken from the  literature$\ci$te{marrone2018}.
The SED of $\targname$ falls between these two categories of the dusty starburst and the blue quasar, representing a transient phase between them.
\label{fig:full_sed} (*fig:full_sed*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

18  publications in the last 7 days.
	 _build/html/2204.06393.md
	 _build/html/2204.03335.md
	 _build/html/2204.03253.md
	 _build/html/2204.02998.md
	 _build/html/2204.02109.md
	 _build/html/2204.02017.md
	 _build/html/2204.01824.md
	 _build/html/2204.01758.md
	 _build/html/2204.01245.md
	 _build/html/2204.00793.md
	 _build/html/2204.00342.md
	 _build/html/2203.16959.md
	 _build/html/2203.16856.md
	 _build/html/2203.16735.md
	 _build/html/2203.16734.md
	 _build/html/2203.16504.md
	 _build/html/2203.15822.md
	 _build/html/2203.15811.md


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers